# Basic Hypergeometric Summation

A **basic hypergeometric series** (or $q$-hypergeometric series) is defined as:

$${}_r\phi_s\left(\begin{matrix} a_1, a_2, \ldots, a_r \\ b_1, b_2, \ldots, b_s \end{matrix}; q, z\right) = \sum_{n=0}^{\infty} \frac{(a_1;q)_n (a_2;q)_n \cdots (a_r;q)_n}{(b_1;q)_n (b_2;q)_n \cdots (b_s;q)_n} \frac{z^n}{(q;q)_n}$$

where $(a;q)_n = \prod_{k=0}^{n-1}(1 - aq^k)$ is the $q$-Pochhammer symbol.

q-Kangaroo can evaluate these series, apply summation formulas, and perform
classical transformations. Parameters are specified as tuples `(num, den, power)`
representing the $q$-monomial $\frac{\text{num}}{\text{den}} \cdot q^{\text{power}}$.

In [1]:
from q_kangaroo import QSession, phi

s = QSession()

# Evaluate 2phi1(q^2, q^3; q^5; q, q)
# upper = [q^2, q^3], lower = [q^5], z = q
result = phi(s, [(1,1,2), (1,1,3)], [(1,1,5)], 1, 1, 1, 20)
result

1 + q + 2*q^2 + 2*q^3 + 2*q^4 + q^5 + 2*q^6 + q^7 + q^8 + q^9 + q^10 + q^11 + q^12 + q^14 + q^15 + q^16 + q^19 + O(q^20)

## The $q$-Gauss Summation Formula

The **$q$-Gauss formula** (DLMF 17.7.2) provides a closed form for certain ${}_2\phi_1$ series:

$${}_2\phi_1\left(\begin{matrix} a, b \\ c \end{matrix}; q, \frac{c}{ab}\right) = \frac{(c/a;q)_\infty\,(c/b;q)_\infty}{(c;q)_\infty\,(c/(ab);q)_\infty}$$

The `try_summation` function automatically detects when a series matches a known
summation formula and returns the closed form:

In [2]:
from q_kangaroo import try_summation

# 2phi1(q^2, q^3; q^8; q, q^3)  [z = c/(ab) = q^8/(q^2*q^3) = q^3]
closed = try_summation(s, [(1,1,2), (1,1,3)], [(1,1,8)], 1, 1, 3, 20)
if closed is not None:
    print("Closed form found!")
closed

Closed form found!


1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)

## Comparing Direct Evaluation vs Closed Form

We can verify the summation formula by computing the series both ways
and confirming they match:

In [3]:
# Direct evaluation via phi()
direct = phi(s, [(1,1,2), (1,1,3)], [(1,1,8)], 1, 1, 3, 20)
print("Direct: ", direct)
print("Closed: ", closed)
print("Match:", repr(direct) == repr(closed))

Direct:  1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)
Closed:  1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)
Match: True


## Heine's First Transformation

**Heine's first transformation** (DLMF 17.6.1) rewrites a ${}_2\phi_1$ as a
product prefactor times a different ${}_2\phi_1$:

$${}_2\phi_1(a, b; c; q, z) = \frac{(b;q)_\infty (az;q)_\infty}{(c;q)_\infty (z;q)_\infty} \cdot {}_2\phi_1(c/b, z; az; q, b)$$

The `heine1` function returns both the prefactor and the transformed result:

In [4]:
from q_kangaroo import heine1

# Apply Heine's first transformation to the same 2phi1
prefactor, transformed = heine1(s, [(1,1,2), (1,1,3)], [(1,1,8)], 1, 1, 3, 20)
print("Prefactor:  ", prefactor)
print("Transformed:", transformed)

Prefactor:   1 - q^5 - q^6 - q^7 + q^11 + q^12 + q^13 - q^18 + O(q^20)
Transformed: 1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)


The transformed result equals the original series, confirming the identity.
The prefactor is an infinite product involving $q$-Pochhammer symbols.

## References

- DLMF Chapter 17: [q-Hypergeometric and Related Functions](https://dlmf.nist.gov/17)
- G. Gasper and M. Rahman, *Basic Hypergeometric Series*, 2nd ed., Cambridge University Press, 2004